# Download Data

## Download OSM data

In [1]:
import pyrosm
from pyrosm import get_data

fp = get_data("Toronto", directory = "data/osm")

## Download TTC Transit Feed data

In [2]:
import requests

# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "ttc-routes-and-schedules"}
package = requests.get(url, params = params).json()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

# To get metadata for non datastore_active resources:
    if not resource["datastore_active"]:
        url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
        resource_metadata = requests.get(url).json()
        url = resource_metadata["result"]["url"]
        print(resource_metadata)
        # From here, you can use the "url" attribute to download this file


{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=resource_show', 'success': True, 'result': {'cache_last_updated': None, 'cache_url': None, 'created': '2020-12-07T21:50:41.530037', 'datastore_active': False, 'extract_job': 'Airflow: upload_remote_files', 'format': 'ZIP', 'hash': '', 'id': 'cfb6b2b8-6191-41e3-bda1-b175c51148cb', 'is_datastore_cache_file': False, 'is_preview': False, 'last_modified': '2023-09-28T12:00:59', 'metadata_modified': '2023-09-28T13:15:50.193866', 'mimetype': 'application/zip', 'mimetype_inner': None, 'name': 'TTC Routes and Schedules Data', 'package_id': '7795b45e-e65a-4465-81fc-c36b9dfff169', 'position': 0, 'resource_type': None, 'revision_id': '4e00475d-2c1b-421a-ae1c-7686ae77d08f', 'size': 34200336, 'state': 'active', 'url': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/7795b45e-e65a-4465-81fc-c36b9dfff169/resource/cfb6b2b8-6191-41e3-bda1-b175c51148cb/download/opendata_ttc_schedules.zip', 'url_type': 'upload'}}

In [3]:
import requests, zipfile, io

#Download data with API
r = requests.get(url)
with open (r"data/TTC/gtfs.zip", mode="wb") as file:
    file.write(r.content)

#Extract gtfs zipfile
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("analysis/data/ttc")

## Download TTC Subway Lines Shapefiles

In [4]:
import requests

# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "ttc-subway-shapefiles"}
package = requests.get(url, params = params).json()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

    # To get metadata for non datastore_active resources:
    if not resource["datastore_active"]:
        url = base_url + "/api/3/action/resource_show?id=" + resource["id"] #api URL
        resource_metadata = requests.get(url).json()
        url = resource_metadata["result"]["url"] #result url
        print(resource_metadata)
    # From here, you can use the "url" attribute to download this file

{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=resource_show', 'success': True, 'result': {'cache_last_updated': None, 'cache_url': None, 'created': '2019-07-23T18:13:00.742161', 'datastore_active': False, 'format': 'SHP', 'hash': '', 'id': '7d68bb52-3285-45d7-a248-7748cb47f6ce', 'is_datastore_cache_file': False, 'is_preview': False, 'last_modified': '2019-07-23T18:13:00.719515', 'metadata_modified': '2022-03-25T15:06:23.894145', 'mimetype': 'application/zip', 'mimetype_inner': None, 'name': 'ttc-subway-shapefile-wgs84', 'package_id': 'c01c6d71-de1f-493d-91ba-364ce64884ac', 'position': 0, 'resource_type': None, 'revision_id': '279d3841-1011-43db-b225-692623995917', 'size': 20796, 'state': 'active', 'url': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/c01c6d71-de1f-493d-91ba-364ce64884ac/resource/7d68bb52-3285-45d7-a248-7748cb47f6ce/download/ttc-subway-shapefile-wgs84.zip', 'url_type': 'upload'}}


In [5]:
import requests, zipfile, io

r2 = requests.get(url)
z2 = zipfile.ZipFile(io.BytesIO(r2.content))
z2.extractall("analysis/data/ttc")

# Create Network Dataset (travel-times.py script)

In [1]:
%set_env JAVA_HOME=C:\Program Files\Microsoft\jdk-17.0.8.101-hotspot\

env: JAVA_HOME=C:\Program Files\Microsoft\jdk-17.0.8.101-hotspot\


In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import fiona
import r5py
import datetime

C:\Users\teres\AppData\Local\Temp\ipykernel_15124\3027855954.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd
Cannot

In [3]:
#Bring in the geojson files for rinks and hex_grid
rinks = gpd.read_file("data/rinks.geojson")

#Data must contain an ID column
rinks["id"] = rinks._id
hex_grid = gpd.read_file("data/hex-grid-centroids.geojson")

In [4]:
#Create transport network
transport_network = r5py.TransportNetwork("data/osm/Toronto.osm.pbf",["data/ttc/gtfs.zip"])

## Get Minimum Travel Time - Walking + Transit

In [5]:
#Compute travel time matrix for walking and transit on a weekday at 6:30pm
from r5py import TransportMode

travel_time_matrix_computer = r5py.TravelTimeMatrixComputer(
    transport_network,origins=rinks,destinations=hex_grid,
    departure=datetime.datetime(2023,10,10,18,30), 
    transport_modes=[TransportMode.WALK, TransportMode.TRANSIT], speed_walking=4.5)

In [6]:
travel_time_matrix = travel_time_matrix_computer.compute_travel_times()
travel_time_matrix.to_csv("data/travel_time/tt_matrix.csv", index = True)

In [7]:
#Get only minimum travel time to the cloest ring for each hex
import pandas as pd

tt = pd.read_csv("data/travel_time/tt_matrix.csv")
tt["travel_time"] = tt["travel_time"].fillna(120)

#Find minimum walking + transit time from hex to the cloest ring
min_tt_wt_630 = tt.groupby("to_id").agg({"travel_time":[min]})
min_tt_wt_630 = min_tt_wt_630.rename(columns ={"travel_time":"walk_&_transit_weekday"})
min_tt_wt_630.to_csv("data/travel_time/access-min-travel-time.csv")

In [8]:
min_tt_wt_630.head()

,walk_&_transit_weekday
,min
to_id,
1544,43.0
1545,43.0
1708,40.0
1709,41.0
1710,41.0


## Get Minimum Travel Time - Walking

In [9]:
#Compute travel time matrix for walking ONLY 
from r5py import TransportMode

travel_time_matrix_computer_walk = r5py.TravelTimeMatrixComputer(
    transport_network,origins=rinks,destinations=hex_grid,
    departure=datetime.datetime(2023,10,10,18,30),
    transport_modes=[TransportMode.WALK], speed_walking=4.5)

In [10]:
travel_time_matrix_walk = travel_time_matrix_computer_walk.compute_travel_times()
travel_time_matrix_walk.to_csv("data/travel_time/tt_matrix_walk.csv", index = True)

In [11]:
import pandas as pd

tt_walk = pd.read_csv("data/travel_time/tt_matrix_walk.csv")
tt_walk["travel_time"] = tt_walk["travel_time"].fillna(120)

#Find minimum walking time from hex to the cloest ring
min_tt_walk = tt_walk.groupby("to_id").agg({"travel_time":[min]})
min_tt_walk = min_tt_walk.rename(columns ={"travel_time":"walk"})
min_tt_walk.to_csv("data/travel_time/min_walking_travel_time")

In [12]:
min_tt_walk.head()

,walk
,min
to_id,
1544,51.0
1545,51.0
1708,47.0
1709,49.0
1710,51.0


## Get Minimum Travel Time - Transit (Weekday)

In [13]:
#Compute travel time matrix for transit ONLY on a weekday at 6:30pm

travel_time_matrix_computer_transit_weekday = r5py.TravelTimeMatrixComputer(
    transport_network,origins=rinks,destinations=hex_grid,
    departure=datetime.datetime(2023,10,10,18,30),
    transport_modes=[TransportMode.TRANSIT], speed_walking=4.5)

In [14]:
travel_time_matrix_computer_transit_weekday = travel_time_matrix_computer_transit_weekday.compute_travel_times()
travel_time_matrix_computer_transit_weekday.to_csv("data/travel_time/tt_matrix_transit_weekday", index = True)

In [15]:
tt_transit_weekday = pd.read_csv("data/travel_time/tt_matrix_transit_weekday")
tt_transit_weekday["travel_time"] = tt_transit_weekday["travel_time"].fillna(120)

#Find minimum transit time from hex to the cloest ring on a weekday
min_tt_transit_weekday = tt_transit_weekday.groupby("to_id").agg({"travel_time":[min]})
min_tt_transit_weekday = min_tt_transit_weekday.rename(columns = {"travel_time":"transit_weekday"})
min_tt_transit_weekday.to_csv("data/travel_time/min_transit_travel_time")

In [16]:
min_tt_transit_weekday.head()

,transit_weekday
,min
to_id,
1544,43.0
1545,43.0
1708,40.0
1709,41.0
1710,41.0


## Get Minimum Travel Time - Transit (Weekend)

In [17]:
#Compute travel time matrix for transit on a weekend at 2:30pm

travel_time_matrix_computer_transit_weekend = r5py.TravelTimeMatrixComputer(
    transport_network,origins=rinks,destinations=hex_grid,
    departure=datetime.datetime(2023,10,14,14,30), 
    transport_modes=[TransportMode.TRANSIT], speed_walking=4.5)

In [18]:
travel_time_matrix_computer_transit_weekend = travel_time_matrix_computer_transit_weekend.compute_travel_times()
travel_time_matrix_computer_transit_weekend.to_csv("data/travel_time/tt_matrix_transit_weekend", index = True)

In [19]:
tt_matrix_transit_weekend = pd.read_csv("data/travel_time/tt_matrix_transit_weekend")
tt_matrix_transit_weekend["travel_time"] = tt_matrix_transit_weekend["travel_time"].fillna(120)

#Find minimum transit time from hex to the cloest ring on a weekend
min_tt_transit_weekend = tt_matrix_transit_weekend.groupby("to_id").agg({"travel_time":[min]})
min_tt_transit_weekend = min_tt_transit_weekend.rename(columns = {"travel_time":"transit_weekend"})
min_tt_transit_weekend.to_csv("data/travel_time/min_tt_transit_weekend")

In [20]:
min_tt_transit_weekend.head()

,transit_weekend
,min
to_id,
1544,45.0
1545,44.0
1708,41.0
1709,43.0
1710,41.0


## Inner Join to Combine all DFs 

In [21]:
#Use inner join to combine all tables into one df
final = pd.concat([min_tt_wt_630,min_tt_walk,min_tt_transit_weekday,min_tt_transit_weekend],
                 axis=1, join="inner").droplevel(1,axis=1).reset_index(drop=False)
#Export final datatable to a csv
final.to_csv("data/travel_time/travel_time_final.csv")

In [22]:
final

,to_id,walk_&_transit_weekday,walk,transit_weekday,transit_weekend
0,1544,43.0,51.0,43.0,45.0
1,1545,43.0,51.0,43.0,44.0
2,1708,40.0,47.0,40.0,41.0
3,1709,41.0,49.0,41.0,43.0
4,1710,41.0,51.0,41.0,41.0
...,...,...,...,...,...
19583,41369,63.0,120.0,63.0,65.0
19584,41370,67.0,120.0,67.0,69.0
19585,41371,69.0,120.0,69.0,70.0
19586,41532,70.0,120.0,70.0,72.0


## Combine Census Variable DFs

In [59]:
import pandas as pd
import numpy as np

In [60]:
#Data file 1 - Median Income
df1 = pd.read_csv("data/census/data.csv")
df1 = df1.set_index(["GeoUID"]).rename(columns={
    "v_CA21_560: Median total income in 2020 among recipients ($)":"Median Income(2020,$)"})
df1.head()

,Type,Region Name,Area (sq km),Population,Dwellings,Households,"Median Income(2020,$)"
GeoUID,,,,,,,
35180018,DA,35180018,6.1125,1573,540,527,50800.0
35180019,DA,35180019,0.2620,709,284,275,43600.0
35180021,DA,35180021,0.1923,490,231,214,43200.0
35180022,DA,35180022,0.2121,407,206,194,43600.0
35180024,DA,35180024,0.3922,486,173,171,51600.0


In [61]:
#Data file 2 - Immigrant and visble minority population
df2 = pd.read_csv("data/census/data (2).csv")
#Rename columns and remove duplicate columns 
df2 = df2.set_index(["GeoUID"]).rename(columns={
    "v_CA21_4410: Immigrants":"# of Immigrants",
    "v_CA21_4875: Total visible minority population":"# of Visible Minority"})
df2 = df2.drop(columns=["Type", "Region Name", "Area (sq km)", "Population ", "Dwellings ", "Households "])
df2.head()

,# of Immigrants,# of Visible Minority
GeoUID,,
35180018,155.0,110.0
35180019,70.0,50.0
35180021,70.0,15.0
35180022,80.0,50.0
35180024,40.0,0.0


In [62]:
#Data file 3-Population in Low Income Household measured by LIM
df3= pd.read_csv("data/census/data(3).csv")
#Rename column, set GEOUID as index, and remove duplicate columns
df3=df3.set_index(["GeoUID"]).rename(columns={
    "v_CA21_1025: In low income based on the Low-income measure, after tax (LIM-AT)":"Low Income Population"
})
df3=df3.drop(columns=["Type", "Region Name", "Area (sq km)", "Population ", "Dwellings ", "Households "])
df3.head()

,Low Income Population
GeoUID,
35180018,60.0
35180019,70.0
35180021,25.0
35180022,35.0
35180024,20.0


In [63]:
census=pd.concat([df1,df2,df3], axis=1)
census.replace(0, np.nan, inplace=True)

#Calculate immigrant and visible miniority percentages
census["Immigrant%"] = census.apply(lambda x: x["# of Immigrants"]/x["Population "]*100, axis=1)
census["Visible Minority%"] = census.apply(lambda x: x["# of Visible Minority"]/x["Population "]*100, axis =1)
census["Low-Income %"]=census.apply(lambda x:x["Low Income Population"]/x["Population "]*100, axis=1)

#Calculate population density, immigrant density, visible minority density
census["Population Density"] = census.apply(lambda x: x["Population "]/x["Area (sq km)"], axis =1)
census["Immigrant Density"] = census.apply(lambda x: x["# of Immigrants"]/x["Area (sq km)"], axis =1)
census["Visible Minority Density"]= census.apply(lambda x: x["# of Visible Minority"]/x["Area (sq km)"], axis =1)

#Clean Column name
census.columns = census.columns.str.replace(" ", "")
census

#Export as a csv file
census.to_csv("data/census/census.csv")
census

,Type,RegionName,Area(sqkm),Population,Dwellings,Households,"MedianIncome(2020,$)",#ofImmigrants,#ofVisibleMinority,LowIncomePopulation,Immigrant%,VisibleMinority%,Low-Income%,PopulationDensity,ImmigrantDensity,VisibleMinorityDensity
GeoUID,,,,,,,,,,,,,,,,
35180018,DA,35180018,6.1125,1573.0,540.0,527.0,50800.0,155.0,110.0,60.0,9.853783,6.993007,3.814367,257.341513,25.357873,17.995910
35180019,DA,35180019,0.2620,709.0,284.0,275.0,43600.0,70.0,50.0,70.0,9.873061,7.052186,9.873061,2706.106870,267.175573,190.839695
35180021,DA,35180021,0.1923,490.0,231.0,214.0,43200.0,70.0,15.0,25.0,14.285714,3.061224,5.102041,2548.101924,364.014561,78.003120
35180022,DA,35180022,0.2121,407.0,206.0,194.0,43600.0,80.0,50.0,35.0,19.656020,12.285012,8.599509,1918.906176,377.180575,235.737860
35180024,DA,35180024,0.3922,486.0,173.0,171.0,51600.0,40.0,NaN,20.0,8.230453,NaN,4.115226,1239.163692,101.988781,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35431423,DA,35431423,0.8142,2285.0,711.0,706.0,48800.0,510.0,625.0,80.0,22.319475,27.352298,3.501094,2806.435765,626.381724,767.624662
35431425,DA,35431425,5.7717,108.0,46.0,41.0,NaN,20.0,NaN,10.0,18.518519,NaN,9.259259,18.711991,3.465184,NaN
35431426,DA,35431426,0.2193,816.0,345.0,336.0,42400.0,120.0,110.0,70.0,14.705882,13.480392,8.578431,3720.930233,547.195622,501.595987


# Joining DA/Census, Hexagon, Rink, Travel Time data

Calculated average population in each hexagon; calculated #population size closest to each rink

## Calculate the average population in each hexagon

In [70]:
import pandas as pd

In [71]:
#Hexagon-DA join table exported from QGIS
hex_da = pd.read_csv("data/hex_da_join/hex_da_join.csv")
hex_da=hex_da.sort_values(by=["c_RegionName"]).rename(columns={"id":"hex_id"})
hex_da=hex_da.set_index("c_RegionName")
hex_da.head()

,hex_id
c_RegionName,
35180499,38404
35180928,37072
35191295,34942
35200002,34629
35200002,34465


In [72]:
#Find the number of hexagon in each DA polygon
da_count=hex_da.groupby(by=["c_RegionName"]).size()
da_count = da_count.to_frame(name ="count")
da_count.head()

,count
c_RegionName,
35180499,1
35180928,1
35191295,1
35200002,2
35200003,1


In [73]:
#Join the count with the hex-DA join table
hex_da= hex_da.join(da_count, on="c_RegionName",how="left")
hex_da = hex_da.reset_index(drop=False).rename(columns={"c_RegionName":"RegionName"})

#Join the count data with the census table
hex_da = hex_da.join(census.set_index("RegionName"), on="RegionName", validate="m:1")

In [78]:
#Calculate the population size in each of the hexagon
hex_da["Pop_in_Hex"] = hex_da.apply(lambda x:x["Population"]/x["count"], axis=1)

#Export to a csv file
hex_da.to_csv("data/hex_da_join/final_hex_da_census.csv")

## Create rinks-hex-census table

In [84]:
#Table created in QGIS by joining the closest rink to the each hexagon by distance
rinks_hex = pd.read_csv("data/rink_hex_join/rink_hex_join.csv")
rinks_hex = rinks_hex.rename(columns={"id":"hex_id",
                                    "_id":"rink_id"})

#Clean the table by dropping QGIS generated columns
rinks_hex = rinks_hex.drop(columns=['n', 'distance', 'feature_x', 'feature_y', 'nearest_x', 'nearest_y'])

,hex_id,rink_id,Parent Asset Name,Asset ID,Asset Name,Community Council Area,Ward,n,distance,feature_x,feature_y,nearest_x,nearest_y
0,15476,25,Irving W. Chapley Community Centre,43590,Irving W. Chapley Community Centre - Outdoor I...,North York,6.0,1,0.002481,-79.455934,43.763611,-79.455697,43.761141
1,15477,25,Irving W. Chapley Community Centre,43590,Irving W. Chapley Community Centre - Outdoor I...,North York,6.0,1,0.000725,-79.455976,43.761809,-79.455697,43.761141
2,15478,25,Irving W. Chapley Community Centre,43590,Irving W. Chapley Community Centre - Outdoor I...,North York,6.0,1,0.001176,-79.456024,43.760011,-79.455697,43.761141
3,15479,25,Irving W. Chapley Community Centre,43590,Irving W. Chapley Community Centre - Outdoor I...,North York,6.0,1,0.002955,-79.456073,43.758210,-79.455697,43.761141
4,15472,25,Irving W. Chapley Community Centre,43590,Irving W. Chapley Community Centre - Outdoor I...,North York,6.0,1,0.009669,-79.455746,43.770810,-79.455697,43.761141


In [91]:
#Join rink, hexagon, and DA-Census data together
rink_hex_da = pd.read_csv("data/hex_da_join/final_hex_da_census.csv")
rink_hex_da = rink_hex_da.join(rinks_hex.set_index("hex_id"), on="hex_id", how="left").drop(columns=["Unnamed: 0"])
rink_hex_da.head()

,RegionName,hex_id,count,Type,Area(sqkm),Population,Dwellings,Households,"MedianIncome(2020,$)",#ofImmigrants,...,PopulationDensity,ImmigrantDensity,VisibleMinorityDensity,Pop_in_Hex,rink_id,Parent Asset Name,Asset ID,Asset Name,Community Council Area,Ward
0,35180499,38404,1,DA,0.9797,889.0,318.0,304.0,55600.0,275.0,...,907.420639,280.698173,469.531489,889.0,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0
1,35180928,37072,1,DA,23.5528,3799.0,1231.0,1180.0,45600.0,1575.0,...,161.297171,66.871030,124.825923,3799.0,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0
2,35191295,34942,1,DA,14.7069,10353.0,2695.0,2640.0,37600.0,5715.0,...,703.955286,388.593109,645.275347,10353.0,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0
3,35200002,34629,2,DA,0.0504,333.0,106.0,103.0,34000.0,205.0,...,6607.142857,4067.460317,5555.555556,166.5,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0
4,35200002,34465,2,DA,0.0504,333.0,106.0,103.0,34000.0,205.0,...,6607.142857,4067.460317,5555.555556,166.5,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0


## Calculate population closest to each rink

In [131]:
rink_pop = rink_hex_da.groupby(by=["rink_id"]).agg({"Pop_in_Hex":"sum"}).round({"Pop_in_Hex":0})
rink_pop.head()

,Pop_in_Hex
rink_id,
1,275013.0
2,28610.0
3,107450.0
4,54346.0
5,26804.0


## Create dataframe with travel time and Census Variable

In [102]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 50)

In [103]:
travel_time = pd.read_csv("data/travel_time/travel_time_final.csv")
travel_time = travel_time.drop(columns=["Unnamed: 0"]).rename(columns={"to_id":"hex_id"})
travel_time.head()

#Join rink-hex-da dataset with travel time
rink_hex_da = rink_hex_da.join(travel_time.set_index("hex_id"), on="hex_id", how="left")

#Export data as CSV 
rink_hex_da.to_csv("data/travel_time/rink-hex-da_tt_join_final.csv")

#Display the table
rink_hex_da_tt_join = pd.read_csv("data/travel_time/rink-hex-da_tt_join_final.csv")
rink_hex_da_tt_join.head()

,hex_id,walk_&_transit_weekday,walk,transit_weekday,transit_weekend
0,1544,43.0,51.0,43.0,45.0
1,1545,43.0,51.0,43.0,44.0
2,1708,40.0,47.0,40.0,41.0
3,1709,41.0,49.0,41.0,43.0
4,1710,41.0,51.0,41.0,41.0


## Clean DA-Rink table

In [118]:
#Spatial join done in QGIS to join the closest rink to each DA polygon
rink_da = pd.read_csv("data/rinks_da_join/rink_da_join.csv")
rink_da = rink_da.drop(columns=['a', 'q', 't', 'dw', 'hh', 'id', 'pop', 'name', 'rgid', 'rpid', 'ruid',
       'rguid','n', 'distance', 'feature_x', 'feature_y', 'nearest_x',
       'nearest_y'])
rink_da.head()

,c_Type,c_RegionName,c_Area(sqkm),c_Population,c_Dwellings,c_Households,"c_MedianIncome(2020,$)",c_#ofImmigrants,c_#ofVisibleMinority,c_LowIncomePopulation,c_Immigrant%,c_VisibleMinority%,c_Low-Income%,c_PopulationDensity,c_ImmigrantDensity,c_VisibleMinorityDensity,_id,Parent Asset Name,Asset ID,Asset Name,Community Council Area,Ward
0,DA,35180499,0.9797,889.0,318.0,304.0,55600.0,275.0,460.0,35.0,30.933633,51.743532,3.937008,907.420639,280.698173,469.531489,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0
1,DA,35180506,0.9179,1441.0,423.0,414.0,46800.0,580.0,890.0,55.0,40.249827,61.762665,3.816794,1569.887787,631.877111,969.604532,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0
2,DA,35180507,0.9078,912.0,288.0,282.0,52000.0,395.0,505.0,30.0,43.311404,55.372807,3.289474,1004.626570,435.117867,556.289932,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0
3,DA,35180538,0.6728,742.0,257.0,246.0,50400.0,205.0,275.0,35.0,27.628032,37.061995,4.716981,1102.853746,304.696790,408.739596,31,Mccowan District Park,60143,Mccowan District Park - Outdoor Ice Pad,Scarborough,20.0
4,DA,35180541,1.7248,1396.0,435.0,427.0,56400.0,275.0,395.0,25.0,19.699140,28.295129,1.790831,809.369202,159.438776,229.012059,31,Mccowan District Park,60143,Mccowan District Park - Outdoor Ice Pad,Scarborough,20.0


# Achieve 

In [272]:
#Load Hexagon_DA join table - joined using QGIS - potential issue with double counting
hex_da = pd.read_csv("analysis/data/hex_da_join.csv")

#Clean hex_da table
hex_da = hex_da.rename(columns={"id":"hex_id", "c_Region Name": "da_id"})
hex_da = hex_da.astype({"hex_id":"str", "da_id":"str"})

#Prepare a smaller version of the census table keeping only required columns
census_merge = census.drop(columns={'Type',
                                   'RegionName',
                                   'Area(sqkm)'})

In [273]:
#Create table with average travel time by DA - group by da_id
da_travel_time = hex_da.groupby(by=["da_id"], as_index=False).agg({"walk": "mean",
                                                          "walk_&_transit_weekday":"mean",
                                                          "transit_weekday":"mean",
                                                          "transit_weekend":"mean"})

#Change data type of the da_id column
da_travel_time["da_id"] = da_travel_time["da_id"].astype("int32")
#Set that as the index
da_travel_time = da_travel_time.set_index(["da_id"])

In [274]:
#Join average travel time table with census variables
da_travel_time = da_travel_time.join(census_merge, how="left")

da_travel_time.head()

,walk,walk_&_transit_weekday,transit_weekday,transit_weekend,Population,Dwellings,Households,"MedianIncome(2020,$)",#ofImmigrants,#ofVisibleMinority,Immigrant%,VisibleMinority%,PopulationDensity,ImmigrantDensity,VisibleMinorityDensity
da_id,,,,,,,,,,,,,,,
35190261,80.0,36.0,36.0,37.0,7016.0,4264.0,3456.0,41200.0,3050.0,4405.0,43.472064,62.785063,251.515510,109.338983,157.914171
35190338,59.0,36.0,36.0,39.0,1323.0,500.0,487.0,37600.0,815.0,550.0,61.602419,41.572184,5848.806366,3603.006189,2431.476569
35190366,55.0,30.0,30.0,34.0,2219.0,856.0,823.0,37200.0,1200.0,1260.0,54.078414,56.782334,6995.586381,3783.102144,3972.257251
35200002,113.0,40.5,40.5,37.5,333.0,106.0,103.0,34000.0,205.0,280.0,61.561562,84.084084,6607.142857,4067.460317,5555.555556
35200003,113.0,42.0,42.0,39.0,314.0,98.0,88.0,35600.0,205.0,345.0,65.286624,109.872611,6709.401709,4380.341880,7371.794872


In [280]:
#Function to add travel time interval as a new column

def travel(x):
    if 0<=x<15:
        val = "0-15 Mins"
    elif 15<=x<30:
        val = "15-30 Mins"
    elif 30<=x<45:
        val="30-45 Mins"
    elif 45<=x<60:
        val="45-60 Mins"
    elif 60<=x<=120:
        val="60-120 Mins"
    else: 
        val = "N/A"
    return val

#Apply function to da_travel_time dataframe
da_travel_time["walk_time_interval"]=da_travel_time["walk"].apply(travel)

da_travel_time["transit_walk_interval"]=da_travel_time["walk_&_transit_weekday"].apply(travel)

da_travel_time["transit_weekday_interval"]=da_travel_time["transit_weekday"].apply(travel)

da_travel_time["transit_weekend_interval"]=da_travel_time["transit_weekend"].apply(travel)

In [277]:
#Total Population
population_total = da_travel_time["Population"].sum()

#Function to calculate percentage of total population
def percent_population(x):
    percent = x/population_total*100
    return percent

In [295]:
#Calculating number of people living within the respective walk time intervals
walk_percentage = da_travel_time.groupby(by=["walk_time_interval"], as_index=False).agg({"Population":"sum"})
#Calculate percentage
walk_percentage["%Population"]=walk_percentage["Population"].apply(percent_population)

walk_percentage["Method"] = "Walk"
walk_percentage = walk_percentage.rename(columns={"walk_time_interval":"Interval"})

,Interval,Population,%Population,Method
0,0-15 Mins,543537.0,20.311502,Walk
1,15-30 Mins,898463.0,33.574775,Walk
2,30-45 Mins,515997.0,19.282356,Walk
3,45-60 Mins,362116.0,13.531958,Walk
4,60-120 Mins,355893.0,13.299410,Walk


In [296]:
#Calculating number of people living within the respective transit_walk time intervals
transit_walk_percentage = da_travel_time.groupby(by=["transit_walk_interval"], as_index=False).agg({"Population":"sum"})
#Calculate percentage
transit_walk_percentage["%Population"]=transit_walk_percentage["Population"].apply(percent_population)

transit_walk_percentage["Method"] = "Transit_Walk"
transit_walk_percentage = transit_walk_percentage.rename(columns={"transit_walk_interval":"Interval"})


,Interval,Population,%Population,Method
0,0-15 Mins,604683.0,22.596474,Transit_Walk
1,15-30 Mins,1421917.0,53.135793,Transit_Walk
2,30-45 Mins,606309.0,22.657236,Transit_Walk
3,45-60 Mins,39956.0,1.493121,Transit_Walk
4,60-120 Mins,3141.0,0.117376,Transit_Walk


In [301]:
#Calculating number of people living within the respective transit time - weekday
transit_weekday_percentage=da_travel_time.groupby(by=["transit_weekday_interval"], as_index=False).agg({"Population":"sum"})
#Calculate Percentage
transit_weekday_percentage["%Population"]=transit_weekday_percentage["Population"].apply(percent_population)

transit_weekday_percentage["Method"]="Transit_Weekday"
transit_weekday_percentage = transit_weekday_percentage.rename(columns={"transit_weekday_interval":"Interval"})


,Interval,Population,%Population,Method
0,0-15 Mins,604683.0,22.596474,Transit_Weekday
1,15-30 Mins,1421917.0,53.135793,Transit_Weekday
2,30-45 Mins,606309.0,22.657236,Transit_Weekday
3,45-60 Mins,39956.0,1.493121,Transit_Weekday
4,60-120 Mins,3141.0,0.117376,Transit_Weekday


In [303]:
#Calculating number of people living within the respective transit time - weekend
transit_weekend_percentage=da_travel_time.groupby(by=["transit_weekend_interval"], as_index=False).agg({"Population":"sum"})
#Calculate Percentage
transit_weekend_percentage["%Population"]=transit_weekend_percentage["Population"].apply(percent_population)

transit_weekend_percentage["Method"]="Transit_Weekend"
transit_weekend_percentage = transit_weekend_percentage.rename(columns={"transit_weekend_interval":"Interval"})


,Interval,Population,%Population,Method
0,0-15 Mins,592440.0,22.138964,Transit_Weekend
1,15-30 Mins,1388600.0,51.890766,Transit_Weekend
2,30-45 Mins,651050.0,24.329168,Transit_Weekend
3,45-60 Mins,38905.0,1.453846,Transit_Weekend
4,60-120 Mins,5011.0,0.187257,Transit_Weekend


In [309]:
#Combine all %ofPopulation dataframe together

pp_final=pd.concat([walk_percentage,transit_walk_percentage,transit_weekday_percentage,transit_weekend_percentage],
              ignore_index=True)